# Database Storage for CustomData

This notebook demonstrates persistent database storage for custom datasets using SQLite.

## What You'll Learn

1. Creating custom databases
2. Inserting data into databases
3. Loading datasets from databases
4. Querying data efficiently
5. Managing multiple databases

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from zipline.pipeline.data import (
    create_custom_db,
    insert_custom_data,
    from_db,
    query_custom_data,
    list_custom_dbs,
    get_custom_db_info,
    drop_custom_db,
)
from zipline.pipeline import Pipeline

# Use /data directory (mounted volume in Docker)
DB_DIR = "/data/custom_databases"
Path(DB_DIR).mkdir(parents=True, exist_ok=True)

print(f"✓ Using database directory: {DB_DIR}")

## 1. Create a Custom Database

In [ ]:
# Create a database for fundamental data
try:
    db_path = create_custom_db(
        'fundamentals-daily',
        columns={
            'pe_ratio': float,
            'market_cap': float,
            'revenue_growth': float,
            'debt_ratio': float,
        },
        bar_size='1d',
        db_dir=DB_DIR,
    )
    print(f"✓ Created database at: {db_path}")
except Exception as e:
    print(f"Database might already exist: {e}")
    print("Continuing with existing database...")

## 2. Generate and Insert Data

In [ ]:
# Generate sample data
dates = pd.bdate_range('2022-01-01', '2023-12-31')
sids = list(range(1, 21))  # 20 stocks

np.random.seed(42)

print(f"Generating data for {len(dates)} dates and {len(sids)} stocks...")

# Create data in MultiIndex format: (field, sid)
for field in ['pe_ratio', 'market_cap', 'revenue_growth', 'debt_ratio']:
    if field == 'pe_ratio':
        values = np.random.uniform(5, 30, (len(dates), len(sids)))
    elif field == 'market_cap':
        values = np.random.uniform(1e9, 1e11, (len(dates), len(sids)))
    elif field == 'revenue_growth':
        values = np.random.uniform(-0.1, 0.3, (len(dates), len(sids)))
    else:  # debt_ratio
        values = np.random.uniform(0, 2, (len(dates), len(sids)))
    
    # Create DataFrame with MultiIndex columns
    data = pd.DataFrame({
        (field, sid): values[:, i]
        for i, sid in enumerate(sids)
    }, index=dates)
    data.columns = pd.MultiIndex.from_tuples(
        data.columns, names=['field', 'sid']
    )
    
    # Insert data
    insert_custom_data('fundamentals-daily', data, mode='update', db_dir=DB_DIR)
    print(f"  ✓ Inserted {field}")

print("\n✓ Data insertion complete!")

## 3. List and Inspect Databases

In [ ]:
# List all databases
dbs = list_custom_dbs(db_dir=DB_DIR)

print(f"Found {len(dbs)} database(s):\n")

for db in dbs:
    print(f"Database: {db['code']}")
    print(f"  Bar size: {db['bar_size']}")
    print(f"  Columns: {list(db['columns'].keys())}")
    print(f"  Rows: {db['row_count']:,}")
    print(f"  Size: {db['size_mb']:.2f} MB")
    print(f"  Created: {db['created_at']}")
    print()

## 4. Query Data from Database

In [ ]:
# Query all data
all_data = query_custom_data('fundamentals-daily', db_dir=DB_DIR)
print(f"Queried all data: {all_data.shape}")
print(f"\nFirst few rows:")
print(all_data.head(10))

In [ ]:
# Query specific date range
q1_2023 = query_custom_data(
    'fundamentals-daily',
    start_date=pd.Timestamp('2023-01-01'),
    end_date=pd.Timestamp('2023-03-31'),
    db_dir=DB_DIR,
)
print(f"\nQ1 2023 data: {q1_2023.shape}")
print(q1_2023.head())

In [ ]:
# Query specific stocks and columns
specific_query = query_custom_data(
    'fundamentals-daily',
    start_date=pd.Timestamp('2023-06-01'),
    end_date=pd.Timestamp('2023-06-30'),
    sids=[1, 2, 3],
    columns=['pe_ratio', 'revenue_growth'],
    db_dir=DB_DIR,
)
print(f"\nSpecific query (Jun 2023, stocks 1-3, 2 columns): {specific_query.shape}")
print(specific_query.head(10))

## 5. Load Dataset from Database

In [ ]:
# Load dataset from database
Fundamentals = from_db('fundamentals-daily', db_dir=DB_DIR)

print(f"Loaded dataset: {Fundamentals}")
print(f"Dataset name: {Fundamentals.__name__}")
print(f"Columns: {sorted(Fundamentals._column_names)}")
print(f"\nDataset info:")
print(Fundamentals.__doc__)

## 6. Use Database-Backed Data in Pipeline

In [ ]:
# Create a pipeline using database-backed data
value_pipeline = Pipeline(
    columns={
        'pe': Fundamentals.pe_ratio.latest,
        'cap': Fundamentals.market_cap.latest,
        'growth': Fundamentals.revenue_growth.latest,
        'debt': Fundamentals.debt_ratio.latest,
        
        # Value criteria
        'undervalued': Fundamentals.pe_ratio.latest < 15,
        'growing': Fundamentals.revenue_growth.latest > 0.1,
        'stable': Fundamentals.debt_ratio.latest < 1.0,
        
        # Value score
        'value_stock': (
            (Fundamentals.pe_ratio.latest < 15) &
            (Fundamentals.revenue_growth.latest > 0.1) &
            (Fundamentals.debt_ratio.latest < 1.0)
        ),
    },
    screen=Fundamentals.pe_ratio.latest < 20,
)

print("✓ Created pipeline with database-backed data")
print(f"Columns: {list(value_pipeline.columns.keys())}")

# Get the loader
loader = Fundamentals.get_loader()
print(f"\nLoader: {loader}")
print(f"Database path: {loader.db_path}")

## 7. Visualize Database Data

In [ ]:
# Query recent data for visualization
recent_data = query_custom_data(
    'fundamentals-daily',
    start_date=pd.Timestamp('2023-01-01'),
    sids=[1, 2, 3, 4, 5],
    db_dir=DB_DIR,
)

# Pivot for plotting
pe_pivot = recent_data.reset_index().pivot(index='date', columns='sid', values='pe_ratio')
growth_pivot = recent_data.reset_index().pivot(index='date', columns='sid', values='revenue_growth')

# Plot
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

pe_pivot.plot(ax=axes[0], title='PE Ratios Over Time (2023)')
axes[0].set_ylabel('PE Ratio')
axes[0].legend(title='Stock ID')

(growth_pivot * 100).plot(ax=axes[1], title='Revenue Growth % Over Time (2023)')
axes[1].set_ylabel('Growth %')
axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.3)
axes[1].legend(title='Stock ID')

plt.tight_layout()
plt.show()

## 8. Database Performance

Compare query performance for different approaches:

In [ ]:
import time

# Query all data
start = time.time()
all_data = query_custom_data('fundamentals-daily', db_dir=DB_DIR)
time_all = time.time() - start
print(f"Query all data: {time_all:.4f}s ({all_data.shape[0]:,} rows)")

# Query specific date range
start = time.time()
date_range = query_custom_data(
    'fundamentals-daily',
    start_date=pd.Timestamp('2023-01-01'),
    end_date=pd.Timestamp('2023-01-31'),
    db_dir=DB_DIR,
)
time_range = time.time() - start
print(f"Query date range: {time_range:.4f}s ({date_range.shape[0]:,} rows)")

# Query specific stocks
start = time.time()
specific = query_custom_data(
    'fundamentals-daily',
    sids=[1, 2, 3],
    db_dir=DB_DIR,
)
time_specific = time.time() - start
print(f"Query specific sids: {time_specific:.4f}s ({specific.shape[0]:,} rows)")

print(f"\n✓ Filtered queries are {time_all/time_range:.1f}x faster than full queries!")

## Summary

Database storage provides:

- ✅ **Persistent storage** - Data survives across sessions
- ✅ **Efficient querying** - Only load what you need
- ✅ **Scalability** - Handle large datasets
- ✅ **Easy management** - List, inspect, query, delete databases
- ✅ **Incremental updates** - Update existing data without reloading

## Next Steps

- Try the other notebooks for advanced examples
- Read the [Database Storage Guide](../docs/CUSTOM_DATA_DATABASE.md)
- Experiment with your own data!